In [89]:
import geopandas
from bs4 import BeautifulSoup
import pandas
import requests
import time
import random
import numpy

In [90]:
path = geopandas.datasets.get_path("naturalearth_lowres")
world = geopandas.read_file(path)

# Replace country names to match CIA names

In [92]:
country_map = {'United States of America': 'United States',
               'Dem. Rep. Congo': 'Congo, Democratic Republic of the',
               'Dominican Rep.': 'Dominican Republic',
               'Bahamas': 'Bahamas, The',
               "Côte d'Ivoire": "Cote d'Ivoire",
               'Central African Rep.': 'Central African Republic',
               'Congo': 'Congo, Republic of the',
               'Eq. Guinea' : 'Equatorial Guinea',
               'eSwatini': 'Eswatini',
               'Gambia': 'Gambia, The',
               'Myanmar': 'Burma',
               'South Korea': 'Korea, South',
               'Turkey': 'Turkey (Turkiye)',
               'Bosnia and Herz.': 'Bosnia and Herzegovina',
               'S. Sudan': 'South Sudan'}

def replace_names(name):
    if name in country_map.keys():
        name = country_map[name]
    return name

world['name'] = world['name'].map(replace_names).unique()

# Get From CIA

In [93]:
ret = requests.get('https://www.cia.gov/the-world-factbook/references/guide-to-country-comparisons/')
soup = BeautifulSoup(ret.text, 'html.parser')

In [94]:
links = soup.find_all("a", {"class": "link-button bold"})

In [95]:
for link in links:
    name = link.text    
    try:
        table = pandas.read_html('https://www.cia.gov/' + link['href'])[0]
    except ValueError as e:
        print(e)
        continue
    table = table.drop(columns=['Rank', 'Date of Information'] )
    table = table.rename(columns={table.columns[1]: name, 'Country':'name'})
    try:
        world = world.join(table.set_index('name'), on='name')
    except ValueError as e:
        print(e)
        continue            
    time.sleep(random.random())    

No tables found
No tables found
columns overlap but no suffix specified: Index(['Youth unemployment rate (ages 15-24)'], dtype='object')


# Removing dollar symbol

In [ ]:
world = world.fillna(-99)

In [100]:
def remove_dollar(row):
    if isinstance(row, (float, int)):
        return row
    row = row.replace('$','')
    row = row.replace(',','')
    row = row.strip()
    return row

In [101]:
keys = ['Real GDP (purchasing power parity)', 
        'Real GDP per capita', 
        'Current account balance', 
        'Exports', 
        'Imports', 
        'Reserves of foreign exchange and gold', 
        'Debt - external']
for key in keys:
    world[key] = world[key].map(remove_dollar).astype(float)

In [105]:
world.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 55 columns):
 #   Column                                                  Non-Null Count  Dtype   
---  ------                                                  --------------  -----   
 0   pop_est                                                 177 non-null    float64 
 1   continent                                               177 non-null    object  
 2   name                                                    177 non-null    object  
 3   iso_a3                                                  177 non-null    object  
 4   gdp_md_est                                              177 non-null    int64   
 5   geometry                                                177 non-null    geometry
 6   Area                                                    177 non-null    float64 
 7   Population                                              177 non-null    float64 
 8   Median age            

In [106]:
world.to_file('world.gpkg')